In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/02 11:48:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# 1

In [4]:
spark.version

'3.0.3'

In [3]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-16 21:13:50--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.84.132
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.84.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  35.6MB/s    in 21s     

2022-02-16 21:14:11 (34.4 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [4]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [3]:
from pyspark.sql import types

In [4]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [10]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true)))

In [11]:
df = df.repartition(24)

In [12]:
df.write.parquet('fhvhv/2021/02/')

In [12]:
df = spark.read.parquet('fhvhv/2021/02/')

In [13]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



# 2

In [ ]:
# ls -lh
total 208M

In [14]:
df.registerTempTable('hvfhv')

# 3

In [9]:
spark.sql("""
select count(*)
from hvfhv
where date(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



# 4

In [17]:
spark.sql("""
select date(pickup_datetime) as day,  max(unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime)) as longest_trip
from hvfhv
group by 1
order by 2 desc
limit 1
""").show()

+----------+------------+
|       day|longest_trip|
+----------+------------+
|2021-02-11|       75540|
+----------+------------+



# 5

In [18]:
spark.sql("""
select dispatching_base_num, count(*)
from hvfhv
group by 1
order by 2 desc
limit 1
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
+--------------------+--------+



In [10]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-02 11:50:13--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.226.40
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.226.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-03-02 11:50:14 (101 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [22]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [23]:
df_zones.write.parquet('zones', mode='overwrite')

In [24]:
df_zones = spark.read.parquet('zones/')

In [25]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

# 6

In [29]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [50]:
df_temp = df.join(df_zones, df.PULocationID == df_zones.LocationID, how='left')

In [51]:
df_temp_renamed = df_temp \
    .withColumnRenamed('Borough', 'PUBorough') \
    .withColumnRenamed('Zone', 'PUZone') \
    .withColumnRenamed('service_zone', 'PUservice_zone') 

df_temp_renamed

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, LocationID: string, PUBorough: string, PUZone: string, PUservice_zone: string]

In [52]:
df_result = df_temp_renamed.join(df_zones, df_temp_renamed.DOLocationID == df_zones.LocationID, how='left')

df_result_renamed = df_result \
    .withColumnRenamed('Borough', 'DOBorough') \
    .withColumnRenamed('Zone', 'DOZone') \
    .withColumnRenamed('service_zone', 'DOservice_zone') 

In [53]:
df_result_renamed.drop('LocationID').show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+---------+--------------------+--------------+-------------+--------------------+--------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|PUBorough|              PUZone|PUservice_zone|    DOBorough|              DOZone|DOservice_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+---------+--------------------+--------------+-------------+--------------------+--------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|Manhattan|       Midtown North|   Yellow Zone|         null|                null|          null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null| Brooklyn|  Stuyvesant Heights|     Boro Z

In [54]:
df_result_renamed.registerTempTable('hvfhv_zones')

In [55]:
spark.sql("""
select PUZone, DOZone, concat(coalesce(PUZone,'Unknown'), ' / ', coalesce(DOZone,'Unknown')), count(*)
from hvfhv_zones
group by 1,2,3
order by 4 desc
limit 1
""").show()

+-------------------+------+-----------------------------------------------------------------+--------+
|             PUZone|DOZone|concat(coalesce(PUZone, Unknown),  / , coalesce(DOZone, Unknown))|count(1)|
+-------------------+------+-----------------------------------------------------------------+--------+
|Crown Heights North|  null|                                             Crown Heights Nor...|  177801|
+-------------------+------+-----------------------------------------------------------------+--------+

